In [1]:
!pip install openpyxl

     |████████████████████████████████| 242 kB 2.0 MB/s            


In [2]:
import glob
import pandas as pd
import numpy as np
import re
import os
from sklearn.utils import shuffle

In [25]:
os.chdir('..')


In [28]:
columns = ['To Name', 'To Contact', 'To Address']

addition = pd.read_excel('22.8.xlsx',names=columns)


In [29]:
addition.head()

,To Name,To Contact,To Address
0,Nguyễn Thị Như Ý,84963418989,"Thôn Phú Quang xã IahRu' , Huyện Chư Pưh , Gi..."
1,Mã văn lễ,84945444919,"So NHA 40, DUONG 1/5 , Phường Hộ Phòng , Thị ..."
2,Yến Rùa,84846323376,"Xã Tân đông , Xã Tân Đông , Huyện Tân Châu , ..."
3,Vũ Trần Ngọc Hân,84908515830,"029 Lô C2 Chung cư Tân Vĩnh , Phường 6 , Quận..."
4,razido1 (RTS),84522408401,"153/1g tổ 82 ấp tam đông , Xã Thới Tam Thôn ,..."


In [30]:
import numpy as np


isp_maps = {
    70: 'Mobifone',
    79: 'Mobifone',
    77: 'Mobifone',
    76: 'Mobifone',
    78: 'Mobifone',
    32: 'Viettel',
    33: 'Viettel',
    34: 'Viettel',
    35: 'Viettel',
    36: 'Viettel',
    37: 'Viettel',
    38: 'Viettel',
    39: 'Viettel',
    83: 'Vinaphone',
    84: 'Vinaphone',
    85: 'Vinaphone',
    81: 'Vinaphone',
    82: 'Vinaphone',
    97: 'Viettel',
    90: 'Mobifone',
    93: 'Mobifone',
    96: 'Viettel',
    98: 'Viettel',
    91: 'Vinaphone',
    92: 'Vietnammobile',
    94: 'Vinaphone',
    89: 'Mobifone',
    88: 'Vinaphone',
    86: 'Viettel',
    55: 'Vietnammobile',
    56: 'Vietnammobile',
    58: 'Vietnammobile',
    87: 'I-telecom',
    52: 'Vietnammobile'
}


def get_isp_name(maps, string):
    # get isp name from mapping dict
    try:
        return maps.get(int(string[1:3]), np.nan)
    except ValueError:
        return np.nan

In [33]:
def normalize_phones(s: str) -> str:
    if s.startswith('84'):
        return '0' + s[2:]
    elif s.startswith('0'):
        return s[:]
    else:
        return '0' + s

In [34]:
addition['To Contact'] = addition['To Contact'].astype('str').apply(lambda x: normalize_phones(x))

In [36]:
import unicodedata
from typing import List


def check_valid_provinces(list_of_provinces: List[str]) -> str:
    provinces = "Hà Nội  Hà Giang  Cao Bằng  Bắc Kạn  Tuyên Quang\
        Lào Cai  Điện Biên  Lai Châu  Sơn La  Yên Bái\
        Hòa Bình  Thái Nguyên  Lạng Sơn  Quảng Ninh  Bắc Giang\
        Phú Thọ  Vĩnh Phúc  Bắc Ninh  Hải Dương  Hải Phòng\
        Hưng Yên  Thái Bình  Hà Nam  Nam Định  Ninh Bình\
        Thanh Hóa  Nghệ An  Hà Tĩnh  Quảng Bình  Quảng Trị\
        Thừa Thiên Huế  Đà Nẵng  Quảng Nam  Quảng Ngãi  Bình Định\
        Phú Yên  Khánh Hòa  Ninh Thuận  Bình Thuận  Kon Tum  Gia Lai\
        Đắk Lắk  Đắk Nông  Lâm Đồng  Bình Phước  Tây Ninh  Bình Dương\
        Đồng Nai  Bà Rịa Vũng Tàu  Hồ Chí Minh  Long An  Tiền Giang\
        Bến Tre  Trà Vinh  Vĩnh Long  Đồng Tháp  An Giang  Kiên Giang\
        Cần Thơ  Hậu Giang  Sóc Trăng  Bạc Liêu  Cà Mau"
    
    provinces_ascii = unicodedata.normalize('NFKD', provinces.strip()).encode('ascii', 'ignore').decode('ascii')
    provinces = provinces + provinces_ascii
    provinces = ' '.join(provinces.strip().split())
    # print(provinces)
    
    length = len(list_of_provinces)
    last = list_of_provinces[length - 1]
    if last.title() in provinces:
        return last.title()
    else:
        list_of_provinces.pop()
        check_valid_provinces(list_of_provinces)

def location(address: str) -> str:
    
    
    raw = r"h. ch. minh|h. n.i|h. giang|c.o b.ng\
    |b.c k.n|tuy.n quang|l.o cai|.i.n bi.n|lai ch.u|s.n la\
    |y.n b.i|h.. b.nh|th.i nguy.n|l.ng s.n|qu.ng ninh|b.c giang\
    |ph. th.|v.nh ph.c|b.c ninh|h.i d..ng|h.i ph.ng|h.ng y.n\
    |th.i b.nh|h. nam|nam ..nh|ninh b.nh|thanh h..|ngh. an|h. t.nh\
    |qu.ng b.nh|qu.ng tr.|th.a thi.n hu.|.. n.ng|qu.ng nam|qu.ng ng.i\
    |b.nh ..nh|ph. y.n|kh.nh h.a|ninh thu.n|b.nh thu.n|kon tum\
    |gia lai|..k l.k|..k n.ng|l.m ..ng|b.nh ph..c|t.y ninh|b.nh d..ng|..ng nai\
    |b. r.a - v.ng t.u|b. r.a v.ng t.u|long an|ti.n giang\
    |b.n tre|tr. vinh|v.nh long|..ng th.p|an giang|ki.n giang|c.n th.\
    |h.u giang|s.c tr.ng|b.c li.u|c. mau|hn|hcm"
    r = raw.strip().split('|')
    raw = '|(?<=[\s,])'.join(r)
    raw = '(?<=[\s,])' + raw
    
    pattern = re.compile(raw, flags=re.I)
    # print(pattern.pattern)
    
    try:
        # result =  re.search(raw, address, re.IGNORECASE).group()
        results = pattern.findall(address)
        return check_valid_provinces(results)
            
        # return results
    except:
        return 'check lai dia chi'


In [37]:
addition['To Province'] = addition['To Address'].apply(lambda x: location(x))


In [38]:
addition['To District'] = np.nan

,To Name,To Contact,To Address,To Province
0,Nguyễn Thị Như Ý,0963418989,"Thôn Phú Quang xã IahRu' , Huyện Chư Pưh , Gi...",Gia Lai
1,Mã văn lễ,0945444919,"So NHA 40, DUONG 1/5 , Phường Hộ Phòng , Thị ...",Bạc Liêu
2,Yến Rùa,0846323376,"Xã Tân đông , Xã Tân Đông , Huyện Tân Châu , ...",Tây Ninh
3,Vũ Trần Ngọc Hân,0908515830,"029 Lô C2 Chung cư Tân Vĩnh , Phường 6 , Quận...",Hồ Chí Minh
4,razido1 (RTS),0522408401,"153/1g tổ 82 ấp tam đông , Xã Thới Tam Thôn ,...",Hồ Chí Minh


In [40]:
addition['ISP'] = addition['To Contact'].astype('str').apply(lambda x: get_isp_name(isp_maps, x))

In [42]:
addition.head()

,To Name,To Contact,To Address,To Province,ISP
0,Nguyễn Thị Như Ý,0963418989,"Thôn Phú Quang xã IahRu' , Huyện Chư Pưh , Gi...",Gia Lai,Viettel
1,Mã văn lễ,0945444919,"So NHA 40, DUONG 1/5 , Phường Hộ Phòng , Thị ...",Bạc Liêu,Vinaphone
2,Yến Rùa,0846323376,"Xã Tân đông , Xã Tân Đông , Huyện Tân Châu , ...",Tây Ninh,Vinaphone
3,Vũ Trần Ngọc Hân,0908515830,"029 Lô C2 Chung cư Tân Vĩnh , Phường 6 , Quận...",Hồ Chí Minh,Mobifone
4,razido1 (RTS),0522408401,"153/1g tổ 82 ấp tam đông , Xã Thới Tam Thôn ,...",Hồ Chí Minh,Vietnammobile


In [45]:
os.chdir('new-outcome')

In [46]:
files = [f for f in glob.glob('data-*.xlsx')]
files

['data-shuffle-00.xlsx',
 'data-shuffle-01.xlsx',
 'data-shuffle-02.xlsx',
 'data-shuffle-03.xlsx',
 'data-shuffle-04.xlsx',
 'data-shuffle-05.xlsx',
 'data-shuffle-06.xlsx',
 'data-shuffle-07.xlsx',
 'data-shuffle-08.xlsx',
 'data-shuffle-09.xlsx',
 'data-shuffle-10.xlsx',
 'data-shuffle-11.xlsx',
 'data-shuffle-12.xlsx',
 'data-shuffle-13.xlsx',
 'data-shuffle-14.xlsx',
 'data-shuffle-15.xlsx',
 'data-shuffle-16.xlsx',
 'data-shuffle-17.xlsx',
 'data-shuffle-18.xlsx',
 'data-shuffle-19.xlsx',
 'data-shuffle-20.xlsx',
 'data-shuffle-21.xlsx',
 'data-shuffle-22.xlsx',
 'data-shuffle-23.xlsx',
 'data-shuffle-24.xlsx',
 'data-shuffle-25.xlsx',
 'data-shuffle-26.xlsx',
 'data-shuffle-27.xlsx',
 'data-shuffle-28.xlsx',
 'data-shuffle-29.xlsx',
 'data-shuffle-30.xlsx',
 'data-shuffle-31.xlsx',
 'data-shuffle-32.xlsx',
 'data-shuffle-33.xlsx',
 'data-shuffle-34.xlsx',
 'data-shuffle-35.xlsx',
 'data-shuffle-36.xlsx',
 'data-shuffle-37.xlsx',
 'data-shuffle-38.xlsx',
 'data-shuffle-39.xlsx',


In [15]:
# files.remove('data-00.xlsx')
# files.remove('data-01.xlsx')
# files

In [47]:
data = [pd.read_excel(file, converters={'To Contact': str}) for file in files]  # read excel do not parse number



In [49]:
data = [df.drop(['sent'], axis=1) for df in data]

In [52]:
total = pd.concat(data)

In [54]:
total.drop_duplicates(subset=['To Contact'], inplace=True)

In [55]:
total.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1174575 entries, 0 to 7021
Data columns (total 6 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   To Name      1174569 non-null  object
 1   To Contact   1174575 non-null  object
 2   To Address   1174575 non-null  object
 3   To Province  1104135 non-null  object
 4   To District  451420 non-null   object
 5   ISP          1173118 non-null  object
dtypes: object(6)
memory usage: 62.7+ MB


In [57]:
total = pd.concat([total, addition])

In [59]:
total['To Contact'].is_unique 

False

In [60]:
total.drop_duplicates(subset=['To Contact'], inplace=True)

In [61]:
total['To Contact'].is_unique

True

In [63]:
total.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1194326 entries, 0 to 23178
Data columns (total 6 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   To Name      1194320 non-null  object
 1   To Contact   1194326 non-null  object
 2   To Address   1194325 non-null  object
 3   To Province  1122698 non-null  object
 4   To District  451420 non-null   object
 5   ISP          1192849 non-null  object
dtypes: object(6)
memory usage: 63.8+ MB


In [64]:
total.head()

,To Name,To Contact,To Address,To Province,To District,ISP
0,Nguyễn Thị Hồng,0395262438,"Vĩnh An Cam Hiếu, Cam Lộ , Xã Cam Hiếu , Huyệ...",Quảng Trị,NaN,Viettel
1,13 Đỗ Thị Hà 200000,0376374191,xã thanh lâm thôn đức hậu huyện mê linh tp hà ...,Hà Nội,NaN,Viettel
2,Hoàng Nguyễn,0943935759,"số A2302, Toà Nhà Riverside, 90 Nguyễn Hưữ Cản...",Hồ Chí Minh,NaN,Vinaphone
3,Vũ Đặng Thiện Hiếu,0349268851,"51B Xuân An, (đi đường Đào Duy Từ, chợ đi xuốn...",Lâm Đồng,NaN,Viettel
4,Lê Hoàng Triều,0707109041,"Ấp lý hoà hiệp , Xã Lý Nhơn , Huyện Cần Giờ ,...",Hồ Chí Minh,NaN,Mobifone


In [65]:
def split_df(df, chunksize=15000):
    chunks = []
    num_chunks = len(df) // chunksize + 1
    for i in range(num_chunks):
        chunks.append(df[i*chunksize:(i+1)*chunksize])
    return chunks

# chunksize = 10000
# chunks = len(data) // chunksize
chunks = split_df(total)

i = 0
for chunk in chunks:
    chunk.to_excel('data-shuffle-{:02d}.xlsx'.format(i), index=False)
    i += 1

In [6]:
# data[1]['To Contact'].is_unique

False

In [10]:
# data = [shuffle(df) for df in data]

In [11]:
# data[1].head()

,To Name,To Contact,To Address,To Province,To District,sent,ISP
4504,Tào Thu,0853168816,Cho e 1 đôi này size 35 Sđt 0853168816 Thôn gi...,Thanh Hóa,Huyện Bá Thước,False,Vinaphone
7709,Yu Miu,0336705687,"47 Âu Cơ , Xã Lộc Nga , Thành Phố Bảo Lộc , L...",Lâm Đồng,NaN,False,Viettel
16,NTD- Chị Thu,0926651631,"23 Đường Suối Nhum - Phường Linh Xuân, Thành p...",Hồ Chí Minh,Thành phố Thủ Đức,False,Vietnammobile
6428,Mạnh Hà,0862176226,"khu vui chơi victory ql 37b tây vinh , Xã Yên...",Nam Định,NaN,False,Viettel
10899,Nguyễn Hoàng Thùy Lâm,0763725149,"Công Viên Diên Hồng, Đường Tản Đà,Phường 1 , ...",Phú Yên,NaN,False,Mobifone


In [12]:
# i = 0
# for df in data:
#     df.to_excel('data-shuffle-{:02d}.xlsx'.format(i), index=False)
#     i += 1